# First tests of inference methods on the epidemiology problem

In [ ]:
import numpy as np
import corner
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

In [7]:
def load_metrics(simulator='epidemiology2d',
                 method='maf',
                 label='',
                 reference_method='maf',
                 reference_label='',
                 single_theta=False):
    
    reference_path = '../data/results/' + simulator + '/' + reference_method + '/'
    path = '../data/results/' + simulator + '/' + method + '/'
    theta_label = '_singletheta' if single_theta else ''
    
    # Reference likelihood
    log_p_references = []
    for run in range(10):
        run_label = '' if run == 0 else '_run_' +  str(run)
        try:
            log_p_references.append(np.load(path + 'log_p_hat' + theta_label + reference_label + run_label + '.npy'))
        except FileNotFoundError:
            pass
    log_p_references = np.array(log_p_references)
    
    print(log_p_references.shape)
    
    log_p_reference = np.mean(log_p_references, axis=1)
    
    # Calculate metrics
    expected_log_ps = []
    roc_aucs = []
    mses = []
    
    for run in range(10):
        run_label = '' if run == 0 else '_run_' +  str(run)
        try:
            log_p = np.load(
                path + 'log_p_hat' + theta_label + label + run_label + '.npy'
            )
            expected_log_ps.append(
                1. / log_p.shape[0] * np.sum(log_p)
            )
            mses.append(
                mean_squared_error(log_likelihood_reference, log_likelihood)
            )
            roc_aucs.append(
                np.load(path + 'roc_auc_surrogate_vs_simulator_' + reference_label + run_label + '.npy')
            )
        
        except FileNotFoundError:
            pass
        
    expected_log_ps = np.array(expected_log_ps)
    roc_aucs = np.array(expected_log_ps)
    mses = np.array(mses)
        
    # Calculate mean and std
    expected_log_p_mean = np.mean(expected_log_ps)
    expected_log_p_uncertainty = np.std(expected_log_ps) / len(expected_log_ps)**0.5
    
    mse_mean = np.mean(mses)
    mse_uncertainty = np.std(msess) / len(mses)**0.5
    
    roc_auc_mean = np.mean(roc_aucs)
    roc_auc_uncertainty = np.std(roc_aucs) / len(roc_aucs)**0.5
    
    return (expected_log_p_mean, expected_log_p_uncertainty,
            mse_mean, mse_uncertainty,
            roc_auc_mean, roc_auc_uncertainty)


## Metrics as function of method and sample size

In [8]:
sample_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]

methods = ['histogram', 'histogram', 'maf', 'maf', 'scandal', 'scandal']
filenames = ['', '_trainedonsingletheta', '', '_trainedonsingletheta', '', '_trainedonsingletheta']
method_labels = ['Histo', 'Histo (trained on 1 theta)', 'MAF', 'MAF (trained on 1 theta)', 'SCANDAL', 'SCANDAL (trained on 1 theta)']


In [9]:
metrics = []

for method, filename in zip(methods, filenames):
    metrics_this_method = []
    
    for sample_size in sample_sizes:
        
        if sample_size == sample_sizes[-1]:
            samplesize_label = '_trainingsamplesize_' + str(sample_size)
        else:
            samplesize_label = ''
            
        metrics_this_method.append(
            load_metrics(
                method=method,
                label= filename + samplesize_label,
                reference_method='maf',
                reference_label='_trainedonsingletheta',
                single_theta=True
            )
        )
        
    metrics.append(metrics_this_method)
    
metrics = np.array(metrics)

expected_log_p_mean = metrics[:,:,0]
expected_log_p_uncertainty = metrics[:,:,1]
mse_mean = metrics[:,:,2]
mse_uncertainty = metrics[:,:,3]
roc_auc_mean = metrics[:,:,4]
roc_auc_uncertainty = metrics[:,:,5]


(0,)


IndexError: tuple index out of range

### Performance vs training sample size

In [ ]:
fig = plt.figure(figsize=(12,4))



ax = plt.subplot(1,3,1)

for m, method in enumerate(method_labels):
    plt.fill_between(
        sample_sizes,
        - expected_log_p_mean[m] - expected_log_p_uncertainty[m],
        - expected_log_p_mean[m] + expected_log_p_uncertainty[m],
        color=colors[m],
        alpha=0.3
    )
    
for m, method in enumerate(method_labels):
    plt.plot(
        sample_sizes,
        - expected_log_likelihoods[m],
        color=colors[m]
        lw=1.5,
        ls='-',
        marker='o',
        ms=5.,
        label=method
    )
    
plt.legend()

plt.xlabel('Training sample size')
plt.ylabel('Negative log likelihood')
ax.set_xscale("log", nonposx='clip')



ax = plt.subplot(1,3,2)

for m, method in enumerate(method_labels):
    plt.fill_between(
        sample_sizes,
        mse_mean[m] - mse_uncertainty[m],
        mse_mean[m] + mse_uncertainty[m],
        color=colors[m],
        alpha=0.3
    )
    
for m, method in enumerate(method_labels):
    plt.plot(
        sample_sizes,
        mse_mean[m],
        color=colors[m]
        lw=1.5,
        ls='-',
        marker='o',
        ms=5.,
        label=method
    )
    
plt.legend()

plt.xlabel('Training sample size')
plt.ylabel('MSE (log likelihood) wrt high-statistics model')
ax.set_xscale("log", nonposx='clip')
ax.set_yscale("log", nonposy='clip')



ax = plt.subplot(1,3,3)

for m, method in enumerate(method_labels):
    plt.fill_between(
        sample_sizes,
        roc_auc_mean[m] - roc_auc_uncertainty[m],
        roc_auc_mean[m] + roc_auc_uncertainty[m],
        color=colors[m],
        alpha=0.3
    )
    
for m, method in enumerate(method_labels):
    plt.plot(
        sample_sizes,
        roc_auc_mean[m],
        color=colors[m]
        lw=1.5,
        ls='-',
        marker='o',
        ms=5.,
        label=method
    )
    
plt.legend()

plt.xlabel('Training sample size')
plt.ylabel('ROC AUC, simulator vs surrogate samples')
ax.set_xscale("log", nonposx='clip')
    
    

plt.tight_layout()
plt.show()


## Original vs simulated samples

In [ ]:
x_labels=['Shannon diversity', '# strains', 'Carriage',
        'Coinfection', 'Prevalence most common strain', '# singletons']
x_ranges = [(0.8,3.0), (3.5,22.5), (0.5, 1.), (0.,0.4), (0.,0.6), (-0.5,12.5)]
x_bins = [20, 19, 10, 7, 12, 12]

x_simulator = np.load('../data/samples/epidemiology/x_test_singletheta.npy')
x_maf = np.load('../data/results/epidemiology/maf/samples_from_p_hat_trainedonsingletheta.npy')

In [ ]:
fig = corner.corner(x_maf,
                   labels=x_labels,
                   range=x_ranges,
                   bins=x_bins,
                   color='C1', alpha=0.5)
_ = corner.corner(x_simulator,
                   fig=fig,
                   labels=x_labels,
                   range=x_ranges,
                   bins=x_bins,
                   color='C0', alpha=0.5)